In [1]:
import pandas as pd
import numpy as np
import datetime as dt

pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)

df = pd.read_csv("datasets/Airbnb_data.csv")

df.head()

In [ ]:
df.shape

(48895, 16)

## 1. Calculating average price

In [ ]:
df['price']

0        149
1        225
2        150
3         89
4         80
        ... 
48890     70
48891     40
48892    115
48893     55
48894     90
Name: price, Length: 48895, dtype: int64

In [ ]:
df['price'].describe()

count    48895.000000
mean       152.720687
std        240.154170
min          0.000000
25%         69.000000
50%        106.000000
75%        175.000000
max      10000.000000
Name: price, dtype: float64

In [ ]:
df['price'].isnull().values.any()

False

In [ ]:
free_listings = df['price'] == 0
temp_df = df[free_listings]
temp_df.head()

id                                               name    host_id  \
23161  18750597  Huge Brooklyn Brownstone Living, Close to it all.    8993084   
25433  20333471      ★Hostel Style Room | Ideal Traveling Buddies★  131697576   
25634  20523843    MARTIAL LOFT 3: REDEMPTION (upstairs, 2nd room)   15787004   
25753  20608117                    Sunny, Quiet Room in Greenpoint    1641537   
25778  20624541      Modern apartment in the heart of Williamsburg   10132166   

          host_name neighbourhood_group       neighbourhood  latitude  \
23161      Kimberly            Brooklyn  Bedford-Stuyvesant  40.69023   
25433        Anisha               Bronx     East Morrisania  40.83296   
25634  Martial Loft            Brooklyn            Bushwick  40.69467   
25753        Lauren            Brooklyn          Greenpoint  40.72462   
25778       Aymeric            Brooklyn        Williamsburg  40.70838   

       longitude        room_type  price  minimum_nights  number_of_reviews  \
23161  -73.95428     Private room      0               4                  1   
25433  -73.88668     Private room      0               2                 55   
25634  -73.92433     Private room      0               2                 16   
25753  -73.94072     Private room      0               2                 12   
25778  -73.94645  Entire home/apt      0               5                  3   

      last_review  reviews_per_month  calculated_host_listings_count  \
23161  2018-01-06               0.05                               4   
25433  2019-06-24               2.56                               4   
25634  2019-05-18               0.71                               5   
25753  2017-10-27               0.53                               2   
25778  2018-01-02               0.15                               1   

       availability_365  
23161                28  
25433               127  
25634                 0  
25753                 0  
25778                73

In [ ]:
temp_df.shape

(11, 16)

In [ ]:
prices= df[~free_listings]['price']
prices.head()

0    149
1    225
2    150
3     89
4     80
Name: price, dtype: int64

In [ ]:
avg_price = round(prices.mean(), 2)
avg_price

152.76

## 2. Comparing costs to the private rental market
<p>Now we know how much a listing costs, on average, per night, but it would be useful to have a benchmark for comparison. 
According to <a href="https://www.zumper.com/rent-research">Zumper</a>, a 1 bedroom apartment in New York City costs, on average, $3,100 per month. Let's convert the per night prices of our listings into monthly costs, so we can compare to the private market. </p>

In [ ]:
price_per_month = prices * 365/12
price_per_month.describe()

count     48884.000000
mean       4646.299522
std        7305.178745
min         304.166667
25%        2098.750000
50%        3224.166667
75%        5322.916667
max      304166.666667
Name: price, dtype: float64

In [ ]:
price_per_month.median()

3224.1666666666665

In [ ]:
round(price_per_month.mean(), 2)

4646.3

## 5. Cleaning the room type column
<p>Unsurprisingly, using Airbnb appears to be substantially more expensive than the private rental market. We should, however, consider that these Airbnb listings include single private rooms or even rooms to share, as well as entire homes/apartments. 
<br><br>
Let's dive deeper into the <code>room_type</code> column to find out the breakdown of listings by type of room. The <code>room_type</code> column has several variations for <code>private room</code> listings, specifically: </p>
<ul>
<li>"Private room"</li>
<li>"private room"</li>
<li>"PRIVATE ROOM"</li>
</ul>
<p>We can solve this by converting all string characters to lower case (upper case would also work just fine). </p>

In [ ]:
df['room_type'].head()

0       Private room
1    Entire home/apt
2       Private room
3    Entire home/apt
4    Entire home/apt
Name: room_type, dtype: object

In [ ]:
room_type = df
room_type['room_type'].replace('NaN', 'shared room', inplace=True)
room_type['room_type'] = room_type['room_type'].str.lower()
room_type['room_type'] = room_type['room_type'].astype('category')
room_type_frequency = room_type['room_type'].value_counts()
room_type_frequency

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3016\2156523523.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  room_type['room_type'].replace('NaN', 'shared room', inplace=True)


room_type
entire home/apt    25409
private room       22326
shared room         1160
Name: count, dtype: int64

## 6. What timeframe are we working with?
<p>It seems there is a fairly similar sized market opportunity for both private rooms (45% of listings) and entire homes/apartments (52%) on the Airbnb platform in NYC.
<br><br></p>
<p>Now let's turn our attention to the <code>reviews</code> DataFrame. The <code>last_review</code> column contains the date of the last review in the format of "Month Day Year" e.g., May 21 2019. We've been asked to find out the earliest and latest review dates in the DataFrame, and ensure the format allows this analysis to be easily conducted going forwards. </p>

In [ ]:
review = df
review['last_review'] = pd.to_datetime(review['last_review'])
null_reviews = review['last_review'].isna()
review = review[~null_reviews]
len(review[review['last_review'].isna()])

# earliest date
earliest_date = review['last_review'].min()

# latest date
latest_date = review['last_review'].max()

print(
    f"Review Analytics:\nEarliest Review: {earliest_date}\nLatest Review: {latest_date}"
)

Review Analytics:
Earliest Review: 2011-03-28 00:00:00
Latest Review: 2019-07-08 00:00:00


## 7. Joining the DataFrames.
<p>Now we've extracted the information needed, we will merge the three DataFrames to make any future analysis easier to conduct. Once we have joined the data, we will remove any observations with missing values and check for duplicates.</p>

In [ ]:
roomsAndPrices = pd.merge(
    df, room_type,
)

In [ ]:
airbnb_merged = pd.merge(
    roomsAndPrices, review,
)
airbnb_merged.dropna(inplace=True)

# Check if there are any duplicate values
print("There are {} duplicates in the DataFrame.".format(airbnb_merged.duplicated().sum()))

There are 0 duplicates in the DataFrame.


In [ ]:
airbnb_merged['last_review'].isna().any()

False

## 8. Analyzing listing prices by NYC borough
<p>Now we have combined all data into a single DataFrame, we will turn our attention to understanding the difference in listing prices between <a href="https://en.wikipedia.org/wiki/Boroughs_of_New_York_City">New York City boroughs</a>. 
We can currently see boroughs listed as the first part of a string within the <code>nbhood_full</code> column, e.g., </p>
<pre><code>Manhattan, Midtown
Brooklyn, Clinton Hill
Manhattan, Murray Hill
Manhattan, Hell's Kitchen
Manhattan, Chinatown</code></pre>
<p></p>
<p>We will therefore need to extract this information from the string and store in a new column, <code>borough</code>, for analysis.</p>

In [ ]:
airbnb_merged.head()

id                                              name  host_id  \
0  2539                Clean & quiet apt home by the park     2787   
1  2595                             Skylit Midtown Castle     2845   
2  3831                   Cozy Entire Floor of Brownstone     4869   
3  5022  Entire Apt: Spacious Studio/Loft by central park     7192   
4  5099         Large Cozy 1 BR Apartment In Midtown East     7322   

     host_name neighbourhood_group neighbourhood  latitude  longitude  \
0         John            Brooklyn    Kensington  40.64749  -73.97237   
1     Jennifer           Manhattan       Midtown  40.75362  -73.98377   
2  LisaRoxanne            Brooklyn  Clinton Hill  40.68514  -73.95976   
3        Laura           Manhattan   East Harlem  40.79851  -73.94399   
4        Chris           Manhattan   Murray Hill  40.74767  -73.97500   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0     private room    149               1                  9  2018-10-19   
1  entire home/apt    225               1                 45  2019-05-21   
2  entire home/apt     89               1                270  2019-07-05   
3  entire home/apt     80              10                  9  2018-11-19   
4  entire home/apt    200               3                 74  2019-06-22   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               0.21                               6               365  
1               0.38                               2               355  
2               4.64                               1               194  
3               0.10                               1                 0  
4               0.59                               1               129

In [ ]:
airbnb_merged['neighbourhood_group']

0             Brooklyn
1            Manhattan
2             Brooklyn
3            Manhattan
4            Manhattan
             ...      
38838        Manhattan
38839           Queens
38840    Staten Island
38841            Bronx
38842         Brooklyn
Name: neighbourhood_group, Length: 38821, dtype: object

## 9. Price range by borough
<p>The above output gives us a summary of prices for listings across the 5 boroughs. In this final task we would like to categorize listings based on whether they fall into specific price ranges, and view this by borough. 
<br><br>
We can do this using percentiles and labels to create a new column, <code>price_range</code>, in the DataFrame.
Once we have created the labels, we can then group the data and count frequencies for listings in each price range by borough.
<br><br>
We will assign the following categories and price ranges:</p>
<table>
<thead>
<tr>
<th>label</th>
<th>price</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>Budget</code></td>
<td>\$0-69</td>
</tr>
<tr>
<td><code>Average</code></td>
<td>\$70-175</td>
</tr>
<tr>
<td><code>Expensive</code></td>
<td>\$176-350</td>
</tr>
<tr>
<td><code>Extravagant</code></td>
<td>&gt; \$350</td>
</tr>
</tbody>
</table>

In [ ]:
label_names = ["Budget", "Average", "Expensive", "Extravagent"]

# Create the label ranges, ranges
ranges = [0, 69, 175, 350, np.inf]
airbnb_merged['price_range'] = pd.cut(airbnb_merged['price'], bins=ranges, labels=label_names)

airbnb= airbnb_merged.groupby(['neighbourhood_group','price_range'])['price_range'].agg("count")
airbnb

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3016\4026464055.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  airbnb= airbnb_merged.groupby(['neighbourhood_group','price_range'])['price_range'].agg("count")


neighbourhood_group  price_range
Bronx                Budget          484
                     Average         352
                     Expensive        30
                     Extravagent       8
Brooklyn             Budget         5253
                     Average        8600
                     Expensive      2180
                     Extravagent     397
Manhattan            Budget         1990
                     Average        8841
                     Expensive      4655
                     Extravagent    1135
Queens               Budget         2106
                     Average        2066
                     Expensive       355
                     Extravagent      45
Staten Island        Budget          141
                     Average         145
                     Expensive        26
                     Extravagent       2
Name: price_range, dtype: int64

In [ ]:
airbnb_merged['price'].dtypes

dtype('int64')